In [6]:
from sage.all import *
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets

# --- Core Function: Generates and plots the Cayley graph ---
# This function incorporates the fixes discussed, including manual vertex labeling
# and refined argument handling for different group types.
def generate_cayley_graph_and_get_plot_data(group_name, group_param, generators_str):
    """
    Generates the Cayley graph for a given finite group and its generators,
    and returns the graph object, group object, and vertex labels for plotting.

    Args:
        group_name (str): The name of the group to create (e.g., "SymmetricGroup", "DihedralGroup").
        group_param: An integer for the group's order (e.g., n for D_n or S_n).
        generators_str (str): A string representing generators (e.g., "(1,2), (1,2,3)").
                              For DihedralGroup, this string will be ignored as default generators are used.

    Returns:
        tuple: (cayley_graph, G, vertex_labels_dict) if successful, None otherwise.
    """
    try:
        G = None
        s_generators = []
        parsed_generators = []
        if generators_str:
            try:
                gen_list_str = f"[{generators_str}]"
                parsed_generators = eval(gen_list_str) # This is where eval() is used.
                if not isinstance(parsed_generators, list) or not all(isinstance(g, tuple) for g in parsed_generators):
                    raise ValueError("Generators must be a comma-separated list of tuples.")
            except Exception as e:
                print(f"Error parsing generators: {e}. Please use format like (1,2), (1,2,3)")
                return None, None, None

        # Determine group type and instantiate it
        if group_name == "DihedralGroup":
            if not isinstance(group_param, int):
                print("Error: For DihedralGroup, the 'group_param' must be an integer 'n'.")
                return None, None, None
            n = group_param
            G = DihedralGroup(n)
            s_generators = [G.gen(0), G.gen(1)] 
            print(f"Dihedral Group D_{n} created with default generators {s_generators}.")

        elif group_name == "SymmetricGroup":
            if not isinstance(group_param, int):
                print("Error: For SymmetricGroup, the 'group_param' must be an integer 'n'.")
                return None, None, None
            if not parsed_generators:
                print("Error: For SymmetricGroup, please provide generators.")
                return None, None, None

            G = SymmetricGroup(group_param)
            print(f"Symmetric Group S_{group_param} created.")
            s_generators = [Permutation(g) for g in parsed_generators]
            print(f"Provided generators: {s_generators}.")

        elif group_name == "AlternatingGroup":
            if not isinstance(group_param, int):
                print("Error: For AlternatingGroup, the 'group_param' must be an integer 'n'.")
                return None, None, None
            if not parsed_generators:
                print("Error: For AlternatingGroup, please provide generators.")
                return None, None, None
            
            G = AlternatingGroup(group_param)
            print(f"Alternating Group A_{group_param} created.")
            s_generators = [Permutation(g) for g in parsed_generators]
            print(f"Provided generators: {s_generators}.")

        else:
            print(f"Error: Group '{group_name}' not supported yet.")
            return None, None, None

        # Check if generators are valid members of the group
        valid_generators = []
        for gen_s in s_generators:
            if gen_s in G:
                valid_generators.append(gen_s)
            else:
                print(f"Warning: Generator {gen_s} is not a valid element of {G}. Skipping.")
        
        if not valid_generators:
            print("No valid generators provided or found. Cannot compute Cayley graph.")
            return None, None, None

        # Compute the Cayley graph
        cayley_graph = G.cayley_graph(generators=valid_generators)
        
        print(f"Cayley Graph for {G} with generators {valid_generators} computed.")
        print(f"Number of vertices: {cayley_graph.order()}")
        print(f"Number of edges: {cayley_graph.size()}")

        # Create vertex labels dictionary
        vertex_labels_dict = {element: str(element) for element in list(G)}
        
        return cayley_graph, G, vertex_labels_dict

    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None, None

# --- Interactive UI Elements ---

# Global variables to store the current graph and group for highlighting
current_cayley_graph = None
current_group = None
current_vertex_labels = {}
current_plot_object = None

# Output widget to display the graph and messages
output_area = widgets.Output(layout=Layout(border='1px solid black', margin='10px 0'))

# Dropdown for group selection
group_selector = widgets.Dropdown(
    options=['SymmetricGroup', 'DihedralGroup', 'AlternatingGroup'],
    value='SymmetricGroup',
    description='Group Type:',
    style={'description_width': 'initial'}
)

# Integer input for group parameter (n)
group_param_input = widgets.IntText(
    value=3,
    description='Group Parameter (n):',
    min=2,
    style={'description_width': 'initial'}
)

# Text area for generators
generators_input = widgets.Textarea(
    value='(1,2), (1,2,3)',
    description='Generators (e.g., (1,2), (1,2,3)):',
    layout=Layout(width='auto', height='80px'),
    style={'description_width': 'initial'}
)

# Button to generate graph
generate_button = widgets.Button(description='Generate Cayley Graph', button_style='primary')

# Dropdown for subgroup selection (will be populated dynamically)
subgroup_selector = widgets.Dropdown(
    options=[],
    description='Highlight Subgroup:',
    disabled=True,
    style={'description_width': 'initial'}
)

# Radio buttons for coset type
coset_type_selector = widgets.RadioButtons(
    options=['None', 'Left Cosets', 'Right Cosets'],
    value='None',
    description='Coset Type:',
    disabled=True,
    style={'description_width': 'initial'}
)

# Button to highlight center
center_button = widgets.Button(description='Highlight Center', disabled=True)

# Button to clear all highlights
clear_highlight_button = widgets.Button(description='Clear Highlights', disabled=True)

# --- Event Handlers ---

def on_generate_button_clicked(b):
    """
    Handles the click event for the 'Generate Cayley Graph' button.
    Generates the graph and updates global variables.
    """
    global current_cayley_graph, current_group, current_vertex_labels, current_plot_object

    with output_area:
        clear_output(wait=True)
        print("Generating graph...")
        
        cayley_graph, G, vertex_labels_dict = generate_cayley_graph_and_get_plot_data(
            group_selector.value,
            group_param_input.value,
            generators_input.value
        )
        
        if cayley_graph is not None:
            current_cayley_graph = cayley_graph
            current_group = G
            current_vertex_labels = vertex_labels_dict

            # Initial plot without highlights - REMOVED spin=True parameter
            current_plot_object = current_cayley_graph.plot(
                layout='spring',
                vertex_size=10,
                vertex_labels=current_vertex_labels,
                graph_border=True,
                # Default vertex colors (all black) - correct format for SageMath
                vertex_colors={'black': list(current_cayley_graph.vertices())}
            )
            current_plot_object.show()  # Removed viewer='tachyon' as it may not be available
            print("Graph generated. Use highlighting options below.")

            # Enable highlighting controls
            subgroup_selector.disabled = False
            coset_type_selector.disabled = False
            center_button.disabled = False
            clear_highlight_button.disabled = False

            # Populate subgroup selector
            populate_subgroup_selector()
        else:
            current_cayley_graph = None
            current_group = None
            current_vertex_labels = {}
            current_plot_object = None
            subgroup_selector.disabled = True
            coset_type_selector.disabled = True
            center_button.disabled = True
            clear_highlight_button.disabled = True

generate_button.on_click(on_generate_button_clicked)

def populate_subgroup_selector():
    """Populates the subgroup dropdown with cyclic subgroups."""
    if current_group:
        # Get all elements of the group
        group_elements = list(current_group)
        # Create options for cyclic subgroups
        subgroup_options = [('None', 'None')] # Default option
        for element in group_elements:
            # Generate cyclic subgroup for each element
            cyclic_subgroup = current_group.subgroup([element])
            # Use a tuple (display_name, value) for dropdown options
            # The value can be the actual subgroup object or its elements
            subgroup_options.append((f"<{element}>", cyclic_subgroup))
        
        subgroup_selector.options = subgroup_options
        subgroup_selector.value = 'None' # Reset to None

def update_plot_with_highlights():
    """Updates the current plot with selected highlighting options."""
    if current_cayley_graph is None or current_group is None:
        return

    # Get all vertices from the graph (these are the group elements themselves)
    all_vertices = list(current_cayley_graph.vertices())
    
    # Start with default colors (all black)
    vertex_colors_map = {v: 'black' for v in all_vertices}
    
    # Get the selected subgroup
    selected_subgroup = subgroup_selector.value
    if selected_subgroup != 'None':
        # Highlight subgroup elements in red
        for element in selected_subgroup:
            # The vertex is the group element itself in Cayley graphs
            if element in vertex_colors_map:
                vertex_colors_map[element] = 'red'

        # Handle coset highlighting
        if coset_type_selector.value != 'None':
            if coset_type_selector.value == 'Left Cosets':
                cosets = current_group.left_cosets(selected_subgroup)
            else: # Right Cosets
                cosets = current_group.right_cosets(selected_subgroup)
            
            # Assign different colors to different cosets
            # Use a fixed set of colors or generate them
            colors = ['blue', 'green', 'purple', 'orange', 'cyan', 'magenta', 'brown', 'pink']
            for i, coset in enumerate(cosets):
                color_index = i % len(colors)
                for element in coset:
                    if element in vertex_colors_map:
                        vertex_colors_map[element] = colors[color_index]
    
    # Highlight center elements in yellow (overrides other colors if applicable)
    if center_button.button_style == 'success' and current_group: # Check if center button is "active"
        center_elements = current_group.center()
        for element in center_elements:
            if element in vertex_colors_map:
                vertex_colors_map[element] = 'yellow'

    # Convert vertex_colors_map to the format expected by SageMath
    # SageMath expects {color: [list_of_vertices]} not {vertex: color}
    sage_vertex_colors = {}
    for vertex, color in vertex_colors_map.items():
        if color not in sage_vertex_colors:
            sage_vertex_colors[color] = []
        sage_vertex_colors[color].append(vertex)

    with output_area:
        clear_output(wait=True)
        if current_cayley_graph:
            # Re-plot with updated colors - REMOVED spin=True parameter
            current_plot_object = current_cayley_graph.plot(
                layout='spring',
                vertex_size=10,
                vertex_labels=current_vertex_labels,
                graph_border=True,
                vertex_colors=sage_vertex_colors # Apply the new colors in correct format
            )
            current_plot_object.show()  # Removed viewer='tachyon'
            print("Graph updated with highlights.")
        else:
            print("No graph to update. Generate a graph first.")

def on_subgroup_selector_changed(change):
    """Handles subgroup selection change."""
    update_plot_with_highlights()

def on_coset_type_selector_changed(change):
    """Handles coset type selection change."""
    update_plot_with_highlights()

def on_center_button_clicked(b):
    """Toggles highlighting of the group center."""
    if center_button.button_style == 'success':
        center_button.button_style = '' # Deactivate
        center_button.description = 'Highlight Center'
    else:
        center_button.button_style = 'success' # Activate
        center_button.description = 'Center Active'
    update_plot_with_highlights()

def on_clear_highlight_button_clicked(b):
    """Clears all highlighting and resets controls."""
    subgroup_selector.value = 'None'
    coset_type_selector.value = 'None'
    center_button.button_style = ''
    center_button.description = 'Highlight Center'
    update_plot_with_highlights() # This will re-plot with default colors

# Attach event handlers
subgroup_selector.observe(on_subgroup_selector_changed, names='value')
coset_type_selector.observe(on_coset_type_selector_changed, names='value')
center_button.on_click(on_center_button_clicked)
clear_highlight_button.on_click(on_clear_highlight_button_clicked)

# --- Layout of the UI ---
ui_elements = widgets.VBox([
    widgets.HBox([group_selector, group_param_input]),
    generators_input,
    generate_button,
    output_area, # Output area for the graph and messages
    widgets.HBox([subgroup_selector, coset_type_selector]),
    widgets.HBox([center_button, clear_highlight_button])
])

display(ui_elements)